In [1]:
%run ../notebooks/01_data_and_utils.ipynb

[INFO] Using device: cuda
[INFO] Directory structure initialized
✅ GoPro dataset ready.
✅ RealBlur dataset ready.
✅ LOL dataset ready.
✅ ICDAR 2015 dataset ready.

❗ RailSem19 NOT found.
Download from: https://www.railsense.org/datasets/railsem19
Extract into: C:\Users\Swayam\OneDrive\Desktop\Adani - RailVision\data\datasets\RailSem19


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim


In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Using device:", DEVICE)


[INFO] Using device: cuda


In [4]:
# =====================
# DATASET PATHS
# =====================
GOPRO_DIR = Path("../data/datasets/GoPro")

TRAIN_BLUR = GOPRO_DIR / "train/blur"
TRAIN_SHARP = GOPRO_DIR / "train/sharp"

VAL_BLUR = GOPRO_DIR / "test/blur"
VAL_SHARP = GOPRO_DIR / "test/sharp"

# =====================
# OUTPUT PATHS
# =====================
MODEL_SAVE_DIR = Path("../outputs/models/deblur")
MODEL_SAVE_DIR.mkdir(parents=True, exist_ok=True)

# =====================
# TRAINING PARAMS
# =====================
BATCH_SIZE = 4          # small batch
NUM_EPOCHS = 200        # long training
LR = 1e-4


In [5]:
def read_image(path):
    img = cv2.imread(str(path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img.astype(np.float32) / 255.0


In [6]:
def to_tensor(img):
    return torch.from_numpy(img).permute(2, 0, 1)



In [7]:
from torch.utils.data import Dataset, DataLoader

class GoProDataset(Dataset):
    def __init__(self, root_dir):
        """
        root_dir = GoPro/train OR GoPro/test
        """
        self.pairs = []

        for seq in sorted(root_dir.iterdir()):
            blur_dir = seq / "blur"
            sharp_dir = seq / "sharp"

            if not blur_dir.exists():
                continue

            blur_imgs = sorted(
                list(blur_dir.glob("*.png")) + list(blur_dir.glob("*.jpg"))
            )

            for b in blur_imgs:
                s = sharp_dir / b.name
                if s.exists():
                    self.pairs.append((b, s))

        if len(self.pairs) == 0:
            raise RuntimeError(f"No image pairs found in {root_dir}")

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        blur_path, sharp_path = self.pairs[idx]

        blur = read_image(blur_path)
        sharp = read_image(sharp_path)

        # 🔥 IMPORTANT: resize (huge memory saver)
        blur = cv2.resize(blur, (256, 256))
        sharp = cv2.resize(sharp, (256, 256))

        blur = to_tensor(blur)
        sharp = to_tensor(sharp)

        return blur, sharp




In [8]:
train_dataset = GoProDataset(GOPRO_DIR / "train")
val_dataset   = GoProDataset(GOPRO_DIR / "test")


train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,   # ✅ FIX
    pin_memory=True
)


val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0
)



In [9]:
class ResBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return x + self.conv2(self.relu(self.conv1(x)))


In [10]:
class DeblurGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.head = nn.Conv2d(3, 64, 7, padding=3)
        self.body = nn.Sequential(*[ResBlock(64) for _ in range(9)])
        self.tail = nn.Conv2d(64, 3, 7, padding=3)

    def forward(self, x):
        x = torch.relu(self.head(x))
        x = self.body(x)
        return torch.sigmoid(self.tail(x))


In [11]:
model = DeblurGenerator().to(DEVICE)

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

# 🔥 IMPORTANT: Mixed Precision + Memory Safety
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))

# Optional but recommended
torch.cuda.empty_cache()


C:\Users\Swayam\AppData\Local\Temp\ipykernel_30112\3753310028.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))


In [12]:
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    psnr_vals, ssim_vals = [], []

    for blur, sharp in loader:
        blur = blur.to(DEVICE)
        sharp = sharp.to(DEVICE)

        out = model(blur)

        out_np = out[0].permute(1,2,0).cpu().numpy()
        sharp_np = sharp[0].permute(1,2,0).cpu().numpy()

        psnr_vals.append(psnr(sharp_np, out_np))
        ssim_vals.append(
            ssim(
                sharp_np,
                out_np,
                channel_axis=2,
                data_range=1.0   # 🔥 FIX
            )
        )


    return np.mean(psnr_vals), np.mean(ssim_vals)


In [13]:
best_psnr = -1
train_losses, val_psnrs = [], []

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0

    for blur, sharp in tqdm(train_loader, desc=f"Epoch {epoch}"):
        blur = blur.to(DEVICE)
        sharp = sharp.to(DEVICE)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            out = model(blur)
            loss = criterion(out, sharp)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)

    val_psnr, val_ssim = evaluate(model, val_loader)
    val_psnrs.append(val_psnr)

    # ---- Saving ----
    save_epoch_model(
        model, optimizer, epoch, epoch_loss, MODEL_SAVE_DIR,
        metrics={"psnr": val_psnr, "ssim": val_ssim}
    )

    save_checkpoint(
        model, optimizer, scheduler, epoch, best_psnr, MODEL_SAVE_DIR
    )

    if val_psnr > best_psnr:
        best_psnr = val_psnr
        save_best_model(model, optimizer, epoch, best_psnr, MODEL_SAVE_DIR)

    scheduler.step()

    print(
        f"[Epoch {epoch}] "
        f"Loss={epoch_loss:.4f} | "
        f"PSNR={val_psnr:.2f} | "
        f"SSIM={val_ssim:.4f}"
    )


Epoch 0:   0%|          | 0/526 [00:00<?, ?it/s]C:\Users\Swayam\AppData\Local\Temp\ipykernel_30112\970465203.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 0:  41%|████      | 215/526 [01:20<01:55,  2.68it/s]


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(train_losses)
plt.title("Training Loss")

plt.subplot(1,2,2)
plt.plot(val_psnrs)
plt.title("Validation PSNR")

plt.show()
